In [ ]:
!git rev-parse HEAD

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pathlib
ROOT = pathlib.Path('..')

In [ ]:
ROOT.absolute()

In [ ]:
import sys
import time
import mlflow
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models
from torchsummary import summary, summary_string # From git only:  pip install git+https://github.com/sksq96/pytorch-summary.git
import matplotlib.pyplot as plt

In [ ]:
sys.path.append(str(ROOT))

In [ ]:
ROOT

In [ ]:
from fed_code.data.dataset import ADNIDataset
from fed_code.data.clients import Clients
from fed_code.model.mrnet import MRNetAlexNet, MRNetResNet50
from fed_code.model.model_io import export_model, import_model
from fed_code.training.hparams import HParams
import fed_code.training.training_blocks as tblocks
from fed_code.training.training_pipes import PipelineCollection
from fed_code.training.tracking import log_string_artifact, log_df_artifact
from fed_code.augmentation.augmentation_for_pretrained import train_transform, test_transform
from fed_code.model.parameter_handling import get_optim_params

In [ ]:
mlflow.set_experiment(experiment_name = "Thesis-main-model")

In [ ]:
data_path = pathlib.Path('/mnt/nvme0n1/ludvig/data/ADNI/ADNI_preprocessed224/')
subject_info_path = pathlib.Path('/mnt/nvme0n1/ludvig/data/ADNI/meta_data/partitioned_participants.csv')
mlflow.log_param('data_path', data_path)
mlflow.log_param('subject_info_path', subject_info_path)

save_path = pathlib.Path('/mnt/nvme0n1/ludvig/models/ADNI/questioning/') / str(time.time())
save_path.mkdir(parents=True, exist_ok=True)
str(save_path)

In [ ]:
'Number of subject folders in path: ', len(list(data_path.glob('sub-*/')))

In [ ]:
subject_info = pd.read_csv(subject_info_path)

In [ ]:
subject_info

In [ ]:
def extract_client_subjects(info_df, id_col, client_col, partition_col, group_col):
    unique_clients = info_df[client_col].unique()
    client_partition_subjects_dict = {}
    for client in unique_clients:
        client_info = info_df[info_df[client_col] == client]
        unique_partitions = client_info[partition_col].unique()
        client_partition_subjects_dict[client] = {}
        for part in unique_partitions:
            part_info = client_info[client_info[partition_col] == part]
            part_ids = list(part_info[id_col])
            part_partitions = list(part_info[partition_col])
            part_groups = list(part_info[group_col])
            client_partition_subjects_dict[client][part] = list(zip(part_ids, part_groups))
    return client_partition_subjects_dict

In [ ]:
client_to_subjects = extract_client_subjects(
    subject_info, 
    id_col='participant_id', 
    client_col='client', 
    partition_col='.partitions', 
    group_col='research_group'
)

In [ ]:
# Create two-way lookups of label->idx->label
label_name_to_idx_dict = {'CN':0, 'MCI':1, 'AD': 2}

In [ ]:
datasets = {}
for client, partitions in client_to_subjects.items():
    print(f'Creating datasets for client: {client}')
    datasets[client] = {}
    for partition, subjects in partitions.items():
        print(f'Loading data for partition: {partition}')
        datasets[client][partition] = ADNIDataset(
            subjects_info=subjects, 
            subject_dirs_path=data_path, 
            labels_dict=label_name_to_idx_dict,
            client=client, 
            is_test=partition in [0, 1]
        )

In [ ]:
clients = Clients()

In [ ]:
# First add local client as this must be the first client added
clients.add_client(training_data=datasets['local'][2], test_data=datasets['local'][1], name='local')
for client_nm, partitions in datasets.items():
    if client_nm not in ['test', 'local']:
        clients.add_client(training_data=partitions[2], test_data=partitions[1], name=client_nm)

In [ ]:
log_string_artifact(str(clients), save_path / 'client-collection-summary.csv')
print(clients)

### Data Statistics
Let's see som statistics for each of the loaded dataset. This is without any augmentation.

In [ ]:
train_data_stats = pd.DataFrame(clients.get_train_statistics()).T
log_df_artifact(train_data_stats, path=save_path / 'training-data-stats.csv')

train_data_stats

In [ ]:
test_data_stats = pd.DataFrame(clients.get_test_statistics()).T
log_df_artifact(test_data_stats, path=save_path / 'test-data-stats.csv')

test_data_stats

## Hyperparameters

In [ ]:
hparams = HParams(
    batch_size=1, 
    test_batch_size=1, 
    epochs_finetune=35, 
    epochs_no_finetune=15,
    label_name_to_idx_dict=label_name_to_idx_dict,
    finetune=True,
    lr_finetune=1e-06,
    lr_no_finetune=1e-06,
    momentum=0.5,
    weight_decay=.01,
    pretrained='resnet50',
    use_cuda=True,
    n_workers=23,
    save_path=save_path, 
    log_interval=30,
    save_model=True,
    dropout_prob=.2,
    aug_center_crop_size = (35, 3, 224, 224),
    aug_max_offset=0.3,
    aug_max_amount_robust_gaussian=0.05,
    aug_max_amount_salt_pepper=0.05,
    aug_max_translate_xy=[15, 15],
    aug_prob_depth_flip=0.5,
    aug_prob_horizontal_flip=0.5,
    aug_prob_vertical_flip=0.5
)

device = 'cuda' if hparams.use_cuda else 'cpu'

hparams.log_params()
mlflow.log_param('device', device)

### Augmentation
Specify the augmentation transformations and assign transformers to the datasets.

In [ ]:
test_transformer = test_transform(
    lowest_allowed=clients.get_min_lower_5_train_value(), 
    highest_allowed=clients.get_max_upper_95_train_value(),
    hparams=hparams
)

train_transformer = train_transform(
    lowest_allowed=clients.get_min_lower_5_train_value(), 
    highest_allowed=clients.get_max_upper_95_train_value(),
    hparams=hparams
)

In [ ]:
clients.set_train_transform(train_transformer)
clients.set_test_transform(test_transformer)

In [ ]:
sample_subject = clients.get_client('client_1').training_data[7]

In [ ]:
sample_subject.keys()

In [ ]:
def plot_subject(subject, num_slices=10, channel_dim=1):
    plt.rcParams["figure.figsize"] = (20, 4)
    img = subject['data']
    mask = subject['brain']
    slices = np.linspace(start=0, stop=img.shape[0]-1, num=num_slices)
    img = img[slices]
    mask = mask[slices]      
    if(len(img.shape)==4):
        num_channels = img.shape[1]
        num_slices = img.shape[0]
        subplot_idx_base = 101 + 10 * (num_channels + 1)
        for sli in range(num_slices):
            plt.figure()
            if sli == 0:
                supt = f'{subject["subject_id"]} {subject["scan_id"]} {subject["label_name"]}'
            else:
                supt = ' '
            plt.suptitle(supt, fontsize=18)     
            for channel in range(num_channels):
                plt.subplot(subplot_idx_base + channel)
                if channel > 0:
                    plt.yticks(color='w')
                # The vmin/max includes the possible offset of 0.5
                # Just a guess though, so TODO
                plt.imshow(img[sli, channel, :,:], vmin=-3.3, vmax=3.3)
            plt.subplot(subplot_idx_base + num_channels)
            plt.yticks(color='w')
            plt.imshow(mask[sli, 0, :,:], vmin=0.0, vmax=1.0)
            plt.tight_layout()
            plt.show()

In [ ]:
sample_subject = clients.get_client('client_1').training_data[7]
plot_subject(sample_subject)

## Modeling

In [ ]:
model_class_dict = {
    'alexnet': MRNetAlexNet,
    'resnet50': MRNetResNet50
}

model_class = model_class_dict[hparams.pretrained]

In [ ]:
model = model_class(num_classes=hparams.num_classes, finetune=hparams.finetune, dropout_prob=hparams.dropout_prob)
model.to(device)
model_summary = summary_string(model, (3,224,224))[0]
log_string_artifact(model_summary, path=save_path / "model_summary.txt")
print(model_summary)

In [ ]:
optimizer_no_finetune = torch.optim.Adam(get_optim_params(model, requires_grad_only=True), hparams.lr_no_finetune, weight_decay=hparams.weight_decay)
optimizer_finetune = torch.optim.Adam(get_optim_params(model, requires_grad_only=False), hparams.lr_finetune, weight_decay=hparams.weight_decay)

# scheduler_no_finetune = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer_no_finetune, patience=hparams.scheduler_patience, 
#     factor=hparams.scheduler_factor, threshold=hparams.scheduler_threshold)
# scheduler_finetune = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer_finetune, patience=hparams.scheduler_patience, 
#     factor=hparams.scheduler_factor, threshold=hparams.scheduler_threshold)

#optimizer_finetune = optim.SGD(get_optim_params(model, requires_grad_only=False), lr=0.001, momentum=0.9)
#optimizer_no_finetune = optim.SGD(get_optim_params(model, requires_grad_only=True), lr=0.001, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.functional.cross_entropy

In [ ]:
training_pipeline_collection = PipelineCollection(
    device=device, 
    hparams=hparams, 
    loss_fn=loss_fn, 
    test_loss_fn=test_loss_fn,
    optimizer_finetune=optimizer_finetune, 
    optimizer_no_finetune=optimizer_no_finetune
)

training_pipe = training_pipeline_collection.local_only(finetune=hparams.finetune)
log_string_artifact(str(training_pipe), save_path / 'training_pipeline.txt')
print(training_pipe)

In [ ]:
global_epoch, model, _ = training_pipe(model, clients, clients, start_epoch=0)

In [ ]:
if hparams.save_model:
    export_model(network_state=model.state_dict(), architecture=model_class, n_classes=hparams.num_classes,
                 inference_transform=test_transformer, model_dir=hparams.save_path)
    model, n_classes, inference_transform = import_model(hparams.save_path)

In [ ]:
#mlflow.end_run()